<a href="https://colab.research.google.com/github/psbruno/Colab/blob/master/Seri%C3%A9_Hist%C3%B3rica_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd
import requests
from datetime import date, timedelta, datetime
import datetime
import os
import html

In [12]:
#função para iterar datas
def daterange(start_date, end_date):
    for i in range(int((end_date - start_date).days)):
        yield start_date + timedelta(i)

In [24]:
def set_file(dist, aplicados, data):
  cod = pd.read_csv("/content/Acento_setado (2).csv").drop(columns='Unnamed: 0').sort_values("Indice c/ acento")
  # dist.to_csv("AAAA/dist.csv")
  #já mandar ordenado
  #aplicados['Código IBGE'] = cod['Código IBGE']
  dist['Código IBGE'] = cod['Código IBGE']
  dist = dist.sort_values("Município").reset_index().drop(columns='index')
  aplicados['Dose'] = aplicados['Dose'].replace("1° Dose","1° DOSE")
  aplicados['Dose'] = aplicados['Dose'].replace("2° Dose","2° DOSE")
  dose1 = aplicados[(aplicados['Dose']=='1° DOSE')].sort_values("Município").reset_index().drop(columns="index")
  
  # aplicados.to_csv('/content/AAA/' + data + '.csv')
  
  dose2 = aplicados[(aplicados['Dose']=='2° DOSE')].sort_values("Município").reset_index().drop(columns="index")
  sum = dose1.drop(columns='Dose')
  sum['Total 1ª Dose'] = dose1['Total Doses Aplicadas']
  sum['Total 2ª Dose'] = dose2['Total Doses Aplicadas']
  sum['Total Doses Aplicadas'] = dose1['Total Doses Aplicadas']+dose2['Total Doses Aplicadas']
  
  
  sum['Vacinas Aplicadas/Distribuidas'] = sum['Total Doses Aplicadas']/dist['Qtd-Doses-Distribuidas']
  sum['Código IBGE'] = cod['Código IBGE']
  #Inserindo população 
  pop = pd.read_csv("IBGE.csv").sort_values("Municipio").reset_index()
  cols= ['Municipio','Populacao_Estimada']
  pop = pop[cols]
  sum['Populacao'] = pop['Populacao_Estimada']

  sum['População Vacinada/População Total'] = sum['Total Doses Aplicadas']/sum['Populacao']
  
  cols = ['Município','Código IBGE','Populacao',"Total 1ª Dose",
          "Total 2ª Dose","Total Doses Aplicadas",'Vacinas Aplicadas/Distribuidas','População Vacinada/População Total']
  
  sum['data'] = data
  sum[cols].to_csv('./diarios/' + data[5:7] + '/' + data[0:10] + '.csv')


In [25]:
import os
from datetime import date, timedelta, datetime
start_date = date(2021, 2, 20)
end_date = date(2021, 5, 13)

for data in daterange(start_date, end_date):
  if(not(os.path.exists('./diarios/' + data.strftime("%m")))):
    os.makedirs('./diarios/' + data.strftime("%m"))

In [34]:
i=0
#dia = 
for data in daterange(start_date, end_date):
  i+=1
  distribuicao = "/content/drive/MyDrive/csv/diariossp/dados/distribuicao/" + data.strftime("%m") + '/' + data.strftime("%Y%m%d") + "_painel_distribuicao_doses.csv"
  vacinometro =  "/content/drive/MyDrive/csv/diariossp/dados/vacinometro/" + data.strftime("%m") + '/' + data.strftime("%Y%m%d") + "_vacinometro.csv"
  if(os.path.exists(distribuicao) and os.path.exists(vacinometro)):
    backup_disp = distribuicao
    backup_vacin = vacinometro
    
    set_file(
    pd.read_csv(distribuicao,sep=';').fillna(0).rename(columns={'Municipio':'Município'}).sort_values("Município").reset_index().drop(columns="index"),
    pd.read_csv(vacinometro,sep=';').fillna(0).rename(columns={'Municipio':'Município', 'Ds Município':'Município'}).sort_values("Município").reset_index().drop(columns="index"),
    str(datetime(data.year, data.month, data.day))
    )
  if(os.path.exists(distribuicao) and not (os.path.exists(vacinometro))):
      backup_disp = distribuicao

      set_file(
      pd.read_csv(distribuicao,sep=';').fillna(0).rename(columns={'Municipio':'Município'}).sort_values("Município").reset_index().drop(columns="index"),
      pd.read_csv(backup_vacin,sep=';').fillna(0).rename(columns={'Municipio':'Município', 'Ds Município':'Município'}).sort_values("Município").reset_index().drop(columns="index"),
      str(datetime(data.year, data.month, data.day))
      )
  if(not os.path.exists(distribuicao) and os.path.exists(vacinometro)):
      backup_vacin = vacinometro
      set_file(
      pd.read_csv(backup_disp,sep=';').fillna(0).rename(columns={'Municipio':'Município'}).sort_values("Município").reset_index().drop(columns="index"),
      pd.read_csv(vacinometro,sep=';').fillna(0).rename(columns={'Municipio':'Município', 'Ds Município':'Município'}).sort_values("Município").reset_index().drop(columns="index"),
      str(datetime(data.year, data.month, data.day))
      )
  if(not os.path.exists(distribuicao) and not os.path.exists(vacinometro)):

      set_file(
      pd.read_csv(backup_disp,sep=';').fillna(0).rename(columns={'Municipio':'Município'}).sort_values("Município").reset_index().drop(columns="index"),
      pd.read_csv(backup_vacin,sep=';').fillna(0).rename(columns={'Municipio':'Município', 'Ds Município':'Município'}).sort_values("Município").reset_index().drop(columns="index"),
      str(datetime(data.year, data.month, data.day))
      )
    

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
